In [2]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLVUHXN/imdb_reviews-train.tfrecord*...…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLVUHXN/imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteLVUHXN/imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
print(info)

tfds.core.DatasetInfo(
    name='imdb_reviews',
    full_name='imdb_reviews/plain_text/1.0.0',
    description="""
    Large Movie Review Dataset.
    This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.
    """,
    config_description="""
    Plain text
    """,
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    data_path='~/tensorflow_datasets/imdb_reviews/plain_text/1.0.0',
    file_format=tfrecord,
    download_size=80.23 MiB,
    dataset_size=129.83 MiB,
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    supervised_keys=('text', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=25000, num_shards=1>,
        '

In [4]:
print(imdb)

{Split('train'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('test'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>, Split('unsupervised'): <PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>}


In [5]:
for example in imdb["train"].take(2):
  print(example)


(<tf.Tensor: shape=(), dtype=string, numpy=b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.">, <tf.Tensor: shape=(), dtype=int64, numpy=0>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on

In [23]:
import numpy as np
train,test=imdb["train"],imdb["test"]

training_sentences=[]
training_labels=[]
testing_sentences=[]
testing_labels=[]

for s,l in train:
  training_sentences.append(s.numpy().decode("utf8"))
  training_labels.append(l.numpy())

for s,l in test:
  testing_sentences.append(s.numpy().decode("utf8"))
  testing_labels.append(l.numpy())

training_labels_final=np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [24]:
vocab_size=10000
max_length=120
embedding_dim=16
trunc_type="post"
oov_tok=""

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [26]:
tokenizer=Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index =tokenizer.word_index

sequences=tokenizer.texts_to_sequences(training_sentences)
padded=pad_sequences(sequences,maxlen=max_length,truncating=trunc_type)

In [27]:
testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length,truncating=trunc_type)

In [28]:
import tensorflow as tf

model=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [29]:
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 11526     
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.fit(padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 6s 6ms/step - loss: 0.4995 - accuracy: 0.7387 - val_loss: 0.3805 - val_accuracy: 0.8288
Epoch 2/10
782/782 [==============================] - 5s 7ms/step - loss: 0.2452 - accuracy: 0.9045 - val_loss: 0.4064 - val_accuracy: 0.8222
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1075 - accuracy: 0.9701 - val_loss: 0.4959 - val_accuracy: 0.8074
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0320 - accuracy: 0.9954 - val_loss: 0.5810 - val_accuracy: 0.8106
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0120 - accuracy: 0.9987 - val_loss: 0.6652 - val_accuracy: 0.8069
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0053 - accuracy: 0.9994 - val_loss: 0.7429 - val_accuracy: 0.8011
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.7782 - val_accuracy: 0.8058
Epoch 

In [32]:
import tensorflow as tf

model1=tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

In [33]:
model1.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [35]:
model1.fit(padded,training_labels_final,epochs=10,validation_data=(testing_padded,testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 6ms/step - loss: 0.5638 - accuracy: 0.7549 - val_loss: 0.4260 - val_accuracy: 0.8178
Epoch 2/10
782/782 [==============================] - 5s 6ms/step - loss: 0.3413 - accuracy: 0.8580 - val_loss: 0.3700 - val_accuracy: 0.8346
Epoch 3/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2755 - accuracy: 0.8878 - val_loss: 0.3677 - val_accuracy: 0.8380
Epoch 4/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2389 - accuracy: 0.9066 - val_loss: 0.3830 - val_accuracy: 0.8325
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.2124 - accuracy: 0.9202 - val_loss: 0.4030 - val_accuracy: 0.8292
Epoch 6/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1913 - accuracy: 0.9310 - val_loss: 0.4273 - val_accuracy: 0.8256
Epoch 7/10
782/782 [==============================] - 5s 6ms/step - loss: 0.1739 - accuracy: 0.9378 - val_loss: 0.4552 - val_accuracy: 0.8215
Epoch 

Usinsg globalpooling1D reduced ovefitting than using flatten

In [36]:
embedding_layer = model.layers[0]

embedding_weights = embedding_layer.get_weights()[0]

print(embedding_weights.shape) 

(10000, 16)


In [37]:
reverse_word_index = tokenizer.index_word

In [38]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):

  word_name = reverse_word_index[word_num]

  word_embedding = embedding_weights[word_num]

  out_m.write(word_name + "\n")

  out_v.write('\t'.join([str(x) for x in word_embedding]) + "\n")

out_v.close()
out_m.close()

In [39]:
try:
  from google.colab import files
except ImportError:
  pass

else:
  files.download('vecs.tsv')
  files.download('meta.tsv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>